In [1]:
from collections import Counter
import numpy as np
import random
from typing import List, Dict

In [2]:
def get_score(motifs: List[List[str]]) -> float:
    '''
    Returns motifs' score
    
        Parameters:
            motifs: motifs array, every line is a DNA string, every cell contains a nucleotide
                    e.g. [['G', 'G', 'C'],
                          ['A', 'A', 'G']]
        Returns:
            score: motif score
    '''
    
    # convert input array into numpy array to simplify the work with columns
    motifs = np.array(motifs)
    n, m = len(motifs), len(motifs[0])
    
    score = 0
    
    for col in range(m):
        # calculate nucleotides counter for one column
        counter = Counter(motifs[:, col])
        max_value = max(counter.values())
        score += n - max_value
        
    return score    

In [3]:
def Random(probabilities: List[float]) -> int:
    ''' Takes in an array with probabilities of getting numbers from [0, len(probabilities)]
            and returns a random number based on probabilities
        For example, probabilities = [0.1, 0.2, 0.3] where 
                     0.1 - probability of getting 0,
                     0.2 - probability of getting 1, etc
    '''

    n = len(probabilities)   
    s = sum(probabilities)
    
    # if probabilities don't sum up to 1 scale the probabilities array
    # for example, [0.1, 0.2, 0.3] -> [0.16, 0.33, 0.49]
    if s != 1:
        probabilities = [p/s for p in probabilities]

    return np.random.choice(np.arange(0, n), p=probabilities)

In [36]:
def get_profile_laplace(motifs: List[List[str]]) -> Dict[str, List[float]]:
    '''         
        Parameters:
            motifs: motifs array, every line is a DNA string, every cell contains a nucleotide
                    e.g. [['G', 'G', 'C'],
                          ['A', 'A', 'G']]
        Returns:
            profile: motifs' profile in dictionary format,
                     incorporating Laplace's Rule of Succession (no 0 probabilities in the answer)
                     e.g. {'A': [0.33, 0.33, 0.16],
                           'C': [0.16, 0.16, 0.33],
                           'G': [0.33, 0.33, 0.33],
                           'T': [0.16, 0.16, 0.16]}
    '''
    
    motifs = np.array(motifs)
    n, m = len(motifs), len(motifs[0])
    
    profile = {
        "A": [1] * m,
        "C": [1] * m,
        "G": [1] * m,
        "T": [1] * m
    }
    
    for col in range(m):
        counter = Counter(motifs[:, col])        
        base = sum(counter.values()) + 4 # we add 1 to each of 4 nucleotides
        
        for nucleotide, arr in profile.items():
            if nucleotide in counter:
                profile[nucleotide][col] += counter[nucleotide]
                profile[nucleotide][col] /= base
            else:
                profile[nucleotide][col] /= base
            
    return profile

In [37]:
def find_kmer_probabilities(dna_string: str, profile: Dict[str, List[float]], k: int) -> List[float]:
    ''' Takes in a single dna string, profile and returns a list of probabilities for all k-mers in the input string '''
    
    n = len(dna_string)

    res = []
    for i in range(n - k + 1):
        k_mer = dna_string[i:i + k]

        prob = 1
        for index, nucleotide in enumerate(k_mer):
            prob *= profile[nucleotide][index]
        
        res.append(prob)

    return res

In [40]:
def GibbsSampler(dna: str, k: int, t: int, max_n: int) -> [List[str], float]:
    ''' Takes in a DNA sequence and returns motifs found by Gibbs sampler
    
    Parameters:
          dna: full DNA string, space separated
            k: lenght of k-mer
            t: number of first DNA lines used for motif search
        max_n: maximum number of searching iterations

    Returns:
            best_motifs: a list of motifs in str format
            motifs_score: corresponding score for the best motifs    
    '''

    def find_best_motifs():    
        best_motifs = []
        best_score = float("inf")

        # randomly select k-mers Motifs = [Motif_1, …, Motif_t] in each string from DNA
        for i in range(t):
            rand_index = random.randint(0, m - k)
            rand_k_mer = dna[i][rand_index:rand_index + k]
            best_motifs.append(rand_k_mer)

        for j in range(max_n):
            # randomly select the i-th row
            i = random.randint(0, t-1)
            
            # get profile for all rows except the i-th
            profile = get_profile_laplace(best_motifs[0:i] + best_motifs[i + 1:])
            # get probabilities for every k-mer in the i-th row based in the profile
            probabilities = find_kmer_probabilities("".join(dna[i]), profile, k)
            # randomly select one k-mer from the i-th row based on the probabilities
            kmer_index = Random(probabilities)
            motifs_i = dna[i][kmer_index:kmer_index + k]
            
            # replace i-th row with the previously selected k-mer
            motifs = best_motifs[0:i] + [motifs_i] + best_motifs[i + 1:]
            motifs_score = get_score(motifs)
            
            # update best motifs and score if needed
            if motifs_score < best_score:
                best_score = motifs_score
                best_motifs = motifs
            
        return best_motifs, best_score
    
    # preprocess input data            
    dna = dna.split(" ")
    dna = [list(row) for row in dna]
    
    n, m = len(dna), len(dna[0])
    
    global_best_motifs = None
    global_best_score = float("inf")

    # run find_best_motifs() function 20 times
    for i in range(20):
        motifs, score = find_best_motifs()

        if score < global_best_score:
            global_best_score = score
            global_best_motifs = motifs
                
    return ["".join(row) for row in global_best_motifs], global_best_score

In [42]:
def get_consensus_string(motifs: List[str]) -> str:
    ''' Takes in motifs array and returns consensus string'''
    
    motifs = [list(row) for row in motifs]
    
    motifs = np.array(motifs)
    n, m = len(motifs), len(motifs[0])
    
    consensus_string = ""
    for col in range(m):
        counter = Counter(motifs[:, col])
        
        nucleotide = max(counter, key=counter.get)
        consensus_string += nucleotide
    
    return consensus_string   

In [41]:
genes = []
with open('data/upstream250.txt') as f:
    genome = f.readlines()
    
    # odd lines - gene tag
    # even lines - actual sequence of 250 nucleotides
    for i in range(1, len(genome), 2):
        # we should remove \n
        genes.append(genome[i][:-1])
        
dna = " ".join(genes)

In [44]:
k, t, max_n = 20, 36, 1000

motif, score = GibbsSampler(dna, k, t, max_n)
consensus_string = get_consensus_string(motif)
print("Score: {} \nConsensus string: {}".format(score, consensus_string))

Score: 252 
 Consensus string: TGGGGACCGTCGGCCCCGGC


In [1]:
len([57, 139, 107, 172, 114, 136, 159, 143, 155, 186, 178, 200, 118, 137, 173, 201, 160, 62, 216, 165, 45, 204])

22

In [2]:
len("TAGGGCCGGAAGTCCCCAAT")

20